# Regression Week 4: Ridge Regression (interpretation)

In this notebook, we will run ridge regression multiple times with different L2 penalties to see which one produces the best fit. We will revisit the example of polynomial regression as a means to see the effect of L2 regularization. In particular, we will:
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression
* Use matplotlib to visualize polynomial regressions
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression, this time with L2 penalty
* Use matplotlib to visualize polynomial regressions under L2 regularization
* Choose best L2 penalty using cross-validation.
* Assess the final fit using test data.

We will continue to use the House data from previous notebooks.  (In the next programming assignment for this module, you will implement your own ridge regression learning algorithm using gradient descent.)

# Fire up graphlab create

In [16]:
import graphlab
graphlab.canvas.set_target('ipynb')

# Polynomial regression, revisited

We build on the material from Week 3, where we wrote the function to produce an SFrame with columns containing the powers of a given input. Copy and paste the function `polynomial_sframe` from Week 3:

In [2]:
def polynomial_sframe(feature, degree):
    # assume that degree >= 1
    # initialize the SFrame:
    poly_sframe = graphlab.SFrame()
    
    # first check if degree > 1
    if degree > 0:
        # then loop over the remaining degrees:
        # range usually starts at 0 and stops at the endpoint-1. We want it to start at 2 and stop at degree
        for power in range(1, degree+1): 
            # first we'll give the column a name:
            name = 'power_' + str(power)
            # then assign poly_sframe[name] to the appropriate power of feature
            poly_sframe[name] = feature.apply(lambda x: x**power)
    return poly_sframe
    

Let's use matplotlib to visualize what a polynomial regression looks like on the house data.

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

This non-commercial license of GraphLab Create for academic use is assigned to lockwm@uw.edu and will expire on May 25, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1498145615.log


In [5]:
sales = graphlab.SFrame('../data/kc_house_data.gl/')

As in Week 3, we will use the sqft_living variable. For plotting purposes (connecting the dots), you'll need to sort by the values of sqft_living. For houses with identical square footage, we break the tie by their prices.

In [6]:
sales = sales.sort(['sqft_living','price'])

Let us revisit the 15th-order polynomial model using the 'sqft_living' input. Generate polynomial features up to degree 15 using `polynomial_sframe()` and fit a model with these features. When fitting the model, use an L2 penalty of `1e-5`:

In [9]:
l2_small_penalty = 1e-5

Note: When we have so many features and so few data points, the solution can become highly numerically unstable, which can sometimes lead to strange unpredictable results.  Thus, rather than using no regularization, we will introduce a tiny amount of regularization (`l2_penalty=1e-5`) to make the solution numerically stable.  (In lecture, we discussed the fact that regularization can also help with numerical stability, and here we are seeing a practical example.)

With the L2 penalty specified above, fit the model and print out the learned weights.

Hint: make sure to add 'price' column to the new SFrame before calling `graphlab.linear_regression.create()`. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set=None` in this call.

In [76]:
def polynomial_model(degree, dataset, l2):
    poly_data = polynomial_sframe(dataset['sqft_living'], degree)
    my_features = poly_data.column_names() # get the name of the features
    poly_data['price'] = dataset['price'] # add price to the data since it's the target
    model = graphlab.linear_regression.create(poly_data, target='price', features=my_features, validation_set=None, l2_penalty=l2)
    print model.get("coefficients").print_rows(num_rows=20)
    # plt.plot(poly_data['power_1'],poly_data['price'],'.', poly_data['power_1'], model.predict(poly_data),'-')
    return model

In [23]:
polynomial_model(15, sales, l2_small_penalty)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.025717     | 2662555.737551     | 245656.462164 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |   167924.862608    |        nan        |
|   power_1   |  None |   103.090936636    |        nan        |
|   power_2   |  None |   0.134604567808   |        nan        |
|   power_3   |  None | -0.000129071373619 |        nan        |
|   power_4   |  None | 5.18928989161e-08  |        nan        |
|   power_5   |  None | -7.77169372788e-12 |        nan        |
|   power_6   |  None | 1.71144963884e-16  |        nan        |
|   power_7   |  None |  4.5117779528e-20  |        nan        |
|   power_8   |  None | -4.78838460102e-25 |        nan        |
|   power_9   |  None | -2.33343496929e-28 |        nan        |
|   power_10  |  None | -7.2902339565e-33  | 1.98223199655e-29 |
|   power_11  |  None | 7.22829349999e-37  |        nan        |
|   power_12  |  None | 6

***QUIZ QUESTION:  What's the learned value for the coefficient of feature `power_1`?***

# Observe overfitting

Recall from Week 3 that the polynomial fit of degree 15 changed wildly whenever the data changed. In particular, when we split the sales data into four subsets and fit the model of degree 15, the result came out to be very different for each subset. The model had a *high variance*. We will see in a moment that ridge regression reduces such variance. But first, we must reproduce the experiment we did in Week 3.

First, split the data into split the sales data into four subsets of roughly equal size and call them `set_1`, `set_2`, `set_3`, and `set_4`. Use `.random_split` function and make sure you set `seed=0`. 

In [11]:
(semi_split1, semi_split2) = sales.random_split(.5,seed=0)
(set_1, set_2) = semi_split1.random_split(0.5, seed=0)
(set_3, set_4) = semi_split2.random_split(0.5, seed=0)

Next, fit a 15th degree polynomial on `set_1`, `set_2`, `set_3`, and `set_4`, using 'sqft_living' to predict prices. Print the weights and make a plot of the resulting model.

Hint: When calling `graphlab.linear_regression.create()`, use the same L2 penalty as before (i.e. `l2_small_penalty`).  Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [24]:
polynomial_model(15, set_1, l2_small_penalty)

Linear regression:

--------------------------------------------------------

Number of examples          : 5404

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.041766     | 2191984.901447     | 248699.117253 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |   9306.46554874    |   719639.980873   |
|   power_1   |  None |    585.86580809    |   2989.33041009   |
|   power_2   |  None |  -0.397305877958   |    5.1431277244   |
|   power_3   |  None | 0.000141470890761  |  0.0048616511985  |
|   power_4   |  None | -1.52945961917e-08 | 2.83657579983e-06 |
|   power_5   |  None | -3.79756708968e-13 | 1.09584374364e-09 |
|   power_6   |  None | 5.97481905044e-17  | 2.97158277304e-13 |
|   power_7   |  None | 1.06888523844e-20  |  5.9463866135e-17 |
|   power_8   |  None | 1.59343810682e-25  | 8.93876714055e-21 |
|   power_9   |  None | -6.92834873348e-29 | 1.00511931468e-24 |
|   power_10  |  None | -6.83813294845e-33 | 8.73365313253e-29 |
|   power_11  |  None | -1.6268620496e-37  | 4.36295862713e-33 |
|   power_12  |  None | 2

In [25]:
polynomial_model(15, set_2, l2_small_penalty)

Linear regression:

--------------------------------------------------------

Number of examples          : 5398

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.051750     | 1975178.191041     | 234533.610646 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |   -25115.8987855   |   1423723.73184   |
|   power_1   |  None |   783.493779662    |   7879.93955583   |
|   power_2   |  None |  -0.767759273168   |   18.4361951306   |
|   power_3   |  None | 0.000438766347065  |  0.0241490759052  |
|   power_4   |  None | -1.15169157765e-07 | 1.97789575952e-05 |
|   power_5   |  None | 6.84281178603e-12  | 1.06841212834e-08 |
|   power_6   |  None | 2.51195186846e-15  | 3.89144841561e-12 |
|   power_7   |  None | -2.06440523307e-19 | 9.51549819948e-16 |
|   power_8   |  None | -4.59673192181e-23 | 1.49496555874e-19 |
|   power_9   |  None | -2.71270417592e-29 | 1.24251826124e-23 |
|   power_10  |  None | 6.21818524969e-31  |        nan        |
|   power_11  |  None | 6.51741334461e-35  | 1.49750520875e-31 |
|   power_12  |  None | -

In [26]:
polynomial_model(15, set_3, l2_small_penalty)

Linear regression:

--------------------------------------------------------

Number of examples          : 5409

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.041422     | 2283722.683416     | 251097.728068 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |   462426.567754    |   1264112.98769   |
|   power_1   |  None |   -759.251854206   |   6160.31746987   |
|   power_2   |  None |   1.02867006808    |   12.6575879778   |
|   power_3   |  None | -0.00052826454477  |  0.0145109950169  |
|   power_4   |  None | 1.15422915761e-07  | 1.03360324025e-05 |
|   power_5   |  None | -2.26096108253e-12 | 4.79760227635e-09 |
|   power_6   |  None | -2.08214269714e-15 | 1.46673002389e-12 |
|   power_7   |  None | 4.08770281089e-20  | 2.86235422373e-16 |
|   power_8   |  None | 2.57079169816e-23  | 3.16708303132e-20 |
|   power_9   |  None |  1.2431125129e-27  | 2.07861266245e-24 |
|   power_10  |  None | -1.72025895153e-31 | 4.61782140639e-28 |
|   power_11  |  None | -2.96761041443e-35 | 5.18146029417e-32 |
|   power_12  |  None | -

In [27]:
polynomial_model(15, set_4, l2_small_penalty)

Linear regression:

--------------------------------------------------------

Number of examples          : 5402

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.045479     | 2378292.371538     | 244341.293202 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |    -170240.0343    |   5295140.21838   |
|   power_1   |  None |   1247.59034572    |   37833.9445817   |
|   power_2   |  None |   -1.22460911405   |    109.69518398   |
|   power_3   |  None | 0.000555254614366  |   0.173719764458  |
|   power_4   |  None | -6.38262301502e-08 | 0.000169409687001 |
|   power_5   |  None | -2.20216009895e-11 | 1.08047762788e-07 |
|   power_6   |  None | 4.81834704182e-15  |  4.6401279293e-11 |
|   power_7   |  None | 4.21461644592e-19  | 1.34934390502e-14 |
|   power_8   |  None | -7.99880720728e-23 | 2.59984285426e-18 |
|   power_9   |  None | -1.32365917399e-26 | 3.08397178281e-22 |
|   power_10  |  None | 1.60197795895e-31  | 1.78718169202e-26 |
|   power_11  |  None | 2.39904346232e-34  | 1.03047979541e-30 |
|   power_12  |  None | 2

The four curves should differ from one another a lot, as should the coefficients you learned.

***QUIZ QUESTION:  For the models learned in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?***  (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Ridge regression comes to rescue

Generally, whenever we see weights change so much in response to change in data, we believe the variance of our estimate to be large. Ridge regression aims to address this issue by penalizing "large" weights. (Weights of `model15` looked quite small, but they are not that small because 'sqft_living' input is in the order of thousands.)

With the argument `l2_penalty=1e5`, fit a 15th-order polynomial model on `set_1`, `set_2`, `set_3`, and `set_4`. Other than the change in the `l2_penalty` parameter, the code should be the same as the experiment above. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [29]:
polynomial_model(15, set_1, 1e5)

Linear regression:

--------------------------------------------------------

Number of examples          : 5404

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.054497     | 5978778.434729     | 374261.720860 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

+-------------+-------+-------------------+-------------------+
|     name    | index |       value       |       stderr      |
+-------------+-------+-------------------+-------------------+
| (intercept) |  None |   530317.024516   |   1082970.05883   |
|   power_1   |  None |   2.58738875673   |   4498.57625495   |
|   power_2   |  None |  0.00127414400592 |   7.73977750303   |
|   power_3   |  None | 1.74934226932e-07 |  0.00731618979541 |
|   power_4   |  None | 1.06022119097e-11 | 4.26869926971e-06 |
|   power_5   |  None | 5.42247604482e-16 | 1.64911065957e-09 |
|   power_6   |  None | 2.89563828343e-20 |  4.4718682342e-13 |
|   power_7   |  None | 1.65000666351e-24 | 8.94858378047e-17 |
|   power_8   |  None | 9.86081528409e-29 |  1.3451750088e-20 |
|   power_9   |  None | 6.06589348254e-33 | 1.51258150225e-24 |
|   power_10  |  None |  3.7891786887e-37 |  1.314307862e-28  |
|   power_11  |  None | 2.38223121312e-41 | 6.56571853519e-33 |
|   power_12  |  None | 1.49847969215e-4

In [30]:
polynomial_model(15, set_2, 1e5)

Linear regression:

--------------------------------------------------------

Number of examples          : 5398

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.051406     | 2984894.541944     | 323238.809634 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

+-------------+-------+-------------------+-------------------+
|     name    | index |       value       |       stderr      |
+-------------+-------+-------------------+-------------------+
| (intercept) |  None |   519216.897383   |   1962203.89503   |
|   power_1   |  None |   2.04470474182   |   10860.2868263   |
|   power_2   |  None |  0.0011314362684  |   25.4091247381   |
|   power_3   |  None | 2.93074277549e-07 |  0.0332827287648  |
|   power_4   |  None | 4.43540598453e-11 | 2.72597462311e-05 |
|   power_5   |  None | 4.80849112204e-15 | 1.47250649326e-08 |
|   power_6   |  None | 4.53091707826e-19 | 5.36327032252e-12 |
|   power_7   |  None | 4.16042910575e-23 | 1.31144457402e-15 |
|   power_8   |  None | 3.90094635128e-27 | 2.06039077436e-19 |
|   power_9   |  None |  3.7773187602e-31 | 1.71246297111e-23 |
|   power_10  |  None | 3.76650326842e-35 |        nan        |
|   power_11  |  None | 3.84228094754e-39 | 2.06389096967e-31 |
|   power_12  |  None | 3.98520828414e-4

In [31]:
polynomial_model(15, set_3, 1e5)

Linear regression:

--------------------------------------------------------

Number of examples          : 5409

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.057911     | 3695342.767093     | 350033.521294 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

+-------------+-------+-------------------+-------------------+
|     name    | index |       value       |       stderr      |
+-------------+-------+-------------------+-------------------+
| (intercept) |  None |   522911.518048   |   1762190.05962   |
|   power_1   |  None |   2.26890421877   |    8587.5632283   |
|   power_2   |  None |  0.00125905041842 |   17.6448434043   |
|   power_3   |  None | 2.77552918155e-07 |  0.0202285170891  |
|   power_4   |  None |  3.2093309779e-11 | 1.44085645294e-05 |
|   power_5   |  None | 2.87573572364e-15 | 6.68792040245e-09 |
|   power_6   |  None | 2.50076112671e-19 | 2.04464086154e-12 |
|   power_7   |  None | 2.24685265906e-23 | 3.99015927317e-16 |
|   power_8   |  None | 2.09349983135e-27 | 4.41495522166e-20 |
|   power_9   |  None | 2.00435383296e-31 | 2.89761327289e-24 |
|   power_10  |  None | 1.95410800249e-35 | 6.43730351532e-28 |
|   power_11  |  None | 1.92734119456e-39 | 7.22302350629e-32 |
|   power_12  |  None | 1.91483699013e-4

In [32]:
polynomial_model(15, set_4, 1e5)

Linear regression:

--------------------------------------------------------

Number of examples          : 5402

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.066136     | 3601895.280124     | 323111.582889 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

+-------------+-------+-------------------+-------------------+
|     name    | index |       value       |       stderr      |
+-------------+-------+-------------------+-------------------+
| (intercept) |  None |   513667.087087   |   7002177.63098   |
|   power_1   |  None |   1.91040938244   |   50030.7809644   |
|   power_2   |  None |  0.00110058029175 |   145.058512488   |
|   power_3   |  None | 3.12753987879e-07 |   0.229723217626  |
|   power_4   |  None | 5.50067886825e-11 | 0.000224023665449 |
|   power_5   |  None | 7.20467557825e-15 | 1.42879998728e-07 |
|   power_6   |  None | 8.24977249384e-19 | 6.13600370367e-11 |
|   power_7   |  None | 9.06503223498e-23 | 1.78434287263e-14 |
|   power_8   |  None | 9.95683160453e-27 | 3.43797533726e-18 |
|   power_9   |  None | 1.10838127982e-30 | 4.07817684548e-22 |
|   power_10  |  None | 1.25315224143e-34 | 2.36332998754e-26 |
|   power_11  |  None | 1.43600781402e-38 | 1.36268394698e-30 |
|   power_12  |  None |  1.662699678e-42

These curves should vary a lot less, now that you applied a high degree of regularization.

***QUIZ QUESTION:  For the models learned with the high level of regularization in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?*** (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Selecting an L2 penalty via cross-validation

Just like the polynomial degree, the L2 penalty is a "magic" parameter we need to select. We could use the validation set approach as we did in the last module, but that approach has a major disadvantage: it leaves fewer observations available for training. **Cross-validation** seeks to overcome this issue by using all of the training set in a smart way.

We will implement a kind of cross-validation called **k-fold cross-validation**. The method gets its name because it involves dividing the training set into k segments of roughtly equal size. Similar to the validation set method, we measure the validation error with one of the segments designated as the validation set. The major difference is that we repeat the process k times as follows:

Set aside segment 0 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
Set aside segment 1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
...<br>
Set aside segment k-1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set

After this process, we compute the average of the k validation errors, and use it as an estimate of the generalization error. Notice that  all observations are used for both training and validation, as we iterate over segments of data. 

To estimate the generalization error well, it is crucial to shuffle the training data before dividing them into segments. GraphLab Create has a utility function for shuffling a given SFrame. We reserve 10% of the data as the test set and shuffle the remainder. (Make sure to use `seed=1` to get consistent answer.)

In [33]:
(train_valid, test) = sales.random_split(.9, seed=1)
train_valid_shuffled = graphlab.toolkits.cross_validation.shuffle(train_valid, random_seed=1)

Once the data is shuffled, we divide it into equal segments. Each segment should receive `n/k` elements, where `n` is the number of observations in the training set and `k` is the number of segments. Since the segment 0 starts at index 0 and contains `n/k` elements, it ends at index `(n/k)-1`. The segment 1 starts where the segment 0 left off, at index `(n/k)`. With `n/k` elements, the segment 1 ends at index `(n*2/k)-1`. Continuing in this fashion, we deduce that the segment `i` starts at index `(n*i/k)` and ends at `(n*(i+1)/k)-1`.

With this pattern in mind, we write a short loop that prints the starting and ending indices of each segment, just to make sure you are getting the splits right.

In [40]:
n = len(train_valid_shuffled)
k = 10 # 10-fold cross-validation

for i in xrange(k):
    start = (n*i)/k
    end = (n*(i+1))/k-1
    exec('validation{0}=train_valid_shuffled[{1}:{2}]'.format(i+1, start, end))

Let us familiarize ourselves with array slicing with SFrame. To extract a continuous slice from an SFrame, use colon in square brackets. For instance, the following cell extracts rows 0 to 9 of `train_valid_shuffled`. Notice that the first index (0) is included in the slice but the last index (10) is omitted.

Now let us extract individual segments with array slicing. Consider the scenario where we group the houses in the `train_valid_shuffled` dataframe into k=10 segments of roughly equal size, with starting and ending indices computed as above.
Extract the fourth segment (segment 3) and assign it to a variable called `validation4`.

To verify that we have the right elements extracted, run the following cell, which computes the average price of the fourth segment. When rounded to nearest whole number, the average should be $536,234.

In [41]:
print int(round(validation4['price'].mean(), 0))

536353


After designating one of the k segments as the validation set, we train a model using the rest of the data. To choose the remainder, we slice (0:start) and (end+1:n) of the data and paste them together. SFrame has `append()` method that pastes together two disjoint sets of rows originating from a common dataset. For instance, the following cell pastes together the first and last two rows of the `train_valid_shuffled` dataframe.

In [ ]:
n = len(train_valid_shuffled)
first_two = train_valid_shuffled[0:2]
last_two = train_valid_shuffled[n-2:n]
print first_two.append(last_two)

Extract the remainder of the data after *excluding* fourth segment (segment 3) and assign the subset to `train4`.

In [48]:
train4 = train_valid_shuffled[:3].append(train_valid_shuffled[4:])

To verify that we have the right elements extracted, run the following cell, which computes the average price of the data with fourth segment excluded. When rounded to nearest whole number, the average should be $539,450.

In [49]:
print int(round(train4['price'].mean(), 0))

539126


Now we are ready to implement k-fold cross-validation. Write a function that computes k validation errors by designating each of the k segments as the validation set. It accepts as parameters (i) `k`, (ii) `l2_penalty`, (iii) dataframe, (iv) name of output column (e.g. `price`) and (v) list of feature names. The function returns the average validation error using k segments as validation sets.

* For each i in [0, 1, ..., k-1]:
  * Compute starting and ending indices of segment i and call 'start' and 'end'
  * Form validation set by taking a slice (start:end+1) from the data.
  * Form training set by appending slice (end+1:n) to the end of slice (0:start).
  * Train a linear model using training set just formed, with a given l2_penalty
  * Compute validation error using validation set just formed

In [73]:
def rss(predictions, outcome):
    # Then compute the residuals/errors
    residuals = outcome - predictions
    # Then square and add them up
    return (residuals * residuals).sum()

def k_fold_cross_validation(k, degree, l2_penalty, data, target, features_list):
    poly_data = polynomial_sframe(data[features_list], degree)
    my_features = poly_data.column_names() # get the name of the features
    poly_data[target] = data[target] # add price to the data since it's the target
    
    index = {}
    for i in xrange(k):
        start = (len(data)*i)/k
        end = (len(data)*(i+1))/k-1
        index[(start, end)] = k
        
    errors = []
    for start, end in sorted(index.keys()):
        model = graphlab.linear_regression.create(poly_data[:start].append(poly_data[end+1:]),
                                                  target=target,
                                                  features=my_features,
                                                  validation_set=poly_data[start:end+1],
                                                  l2_penalty=l2_penalty,
                                                  verbose=False)
        errors += [rss(model.predict(poly_data[start:end+1]), poly_data[start:end+1]['price'])]
    return float(sum(errors)) / len(errors)

Once we have a function to compute the average validation error for a model, we can write a loop to find the model that minimizes the average validation error. Write a loop that does the following:
* We will again be aiming to fit a 15th-order polynomial model using the `sqft_living` input
* For `l2_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, you can use this Numpy function: `np.logspace(1, 7, num=13)`.)
    * Run 10-fold cross-validation with `l2_penalty`
* Report which L2 penalty produced the lowest average validation error.

Note: since the degree of the polynomial is now fixed to 15, to make things faster, you should generate polynomial features in advance and re-use them throughout the loop. Make sure to use `train_valid_shuffled` when generating polynomial features!

In [74]:
import numpy as np
for l2 in np.logspace(1, 7, num=13):
    error = k_fold_cross_validation(10, 15, l2, train_valid_shuffled, 'price', 'sqft_living')
    print l2, error

10.0 4.91826427769e+14
31.6227766017 2.87504229919e+14
100.0 1.60908965822e+14
316.227766017 1.22090967326e+14
1000.0 1.21192264451e+14
3162.27766017 1.2395000929e+14
10000.0 1.36837175248e+14
31622.7766017 1.71728094842e+14
100000.0 2.2936143126e+14
316227.766017 2.52940568729e+14
1000000.0 2.58682548441e+14
3162277.66017 2.62819399742e+14
10000000.0 2.64889015378e+14


***QUIZ QUESTIONS:  What is the best value for the L2 penalty according to 10-fold validation?***

You may find it useful to plot the k-fold cross-validation errors you have obtained to better understand the behavior of the method.  

In [ ]:
# Plot the l2_penalty values in the x axis and the cross-validation error in the y axis.
# Using plt.xscale('log') will make your plot more intuitive.



Once you found the best value for the L2 penalty using cross-validation, it is important to retrain a final model on all of the training data using this value of `l2_penalty`. This way, your final model will be trained on the entire dataset.

In [77]:
model = polynomial_model(15, train_valid, 1000)
rss(model.predict(test), test['price'])

Linear regression:

--------------------------------------------------------

Number of examples          : 19396

Number of features          : 15

Number of unpacked features : 15

...

| 1         | 2        | 0.034734     | 2461778.986191     | 248914.007014 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |   253972.377679    |   540355.503083   |
|   power_1   |  None |   57.3754544277    |   2599.52276213   |
|   power_2   |  None |   0.023783252166   |   5.18937484803   |
|   power_3   |  None | 2.64303484986e-06  |  0.00567944865948 |
|   power_4   |  None | 6.53250411225e-11  | 3.79671555697e-06 |
|   power_5   |  None | -1.0280259983e-15  | 1.63003274977e-09 |
|   power_6   |  None | -1.17062658026e-19 | 4.56643207222e-13 |
|   power_7   |  None | -5.32286483343e-24 | 8.17755871911e-17 |
|   power_8   |  None | -2.90413735131e-28 | 8.47691579883e-21 |
|   power_9   |  None | -2.90869143388e-32 | 2.69917282912e-25 |
|   power_10  |  None | -3.44379026738e-36 | 2.63357767982e-29 |
|   power_11  |  None | -3.74459613949e-40 | 3.44047945722e-33 |
|   power_12  |  None | -

252897427447157.56

***QUIZ QUESTION: Using the best L2 penalty found above, train a model using all training data. What is the RSS on the TEST data of the model you learn with this L2 penalty? ***